In [1]:
%load_ext autoreload
%autoreload 2


In [19]:
import yt

from modules import *
from sim import *
from read_ramses import *


In [41]:
# choose simulation
sim_round = 1
sim_name = "alpha_eps0p1"

sim_dir = os.path.join(sim_base_dir, "round%d" % sim_round, sim_name)

os.chdir(sim_dir)

# choose dump
dump = get_dump(a_exp=0.067)


a_exp = 0.0644


In [42]:
halo = get_biggest_halos(1, dump=dump, use_density=False)
info_file = get_stdout("ls output_%.5d/info*" % dump).split()[0]

if sim_name == "gas":
    halo.coord = np.array([5.105151466E-01, 5.127534424E-01, 4.929928780E-01])
    
print("Halo coords: %.3g, %.3g, %.3g" % (halo.coord[0], halo.coord[1], halo.coord[2]))
    

Halo coords: 0.508, 0.509, 0.496


In [43]:
ds = yt.load(info_file)


yt : [INFO     ] 2023-10-27 16:30:30,811 Parameters: current_time              = 4.991203396022671
yt : [INFO     ] 2023-10-27 16:30:30,812 Parameters: domain_dimensions         = [128 128 128]
yt : [INFO     ] 2023-10-27 16:30:30,812 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2023-10-27 16:30:30,815 Parameters: domain_right_edge         = [1. 1. 1.]
yt : [INFO     ] 2023-10-27 16:30:30,816 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2023-10-27 16:30:30,816 Parameters: current_redshift          = 14.53952658367318
yt : [INFO     ] 2023-10-27 16:30:30,816 Parameters: omega_lambda              = 0.723999977111816
yt : [INFO     ] 2023-10-27 16:30:30,817 Parameters: omega_matter              = 0.275999993085861
yt : [INFO     ] 2023-10-27 16:30:30,817 Parameters: omega_radiation           = 0.0
yt : [INFO     ] 2023-10-27 16:30:30,817 Parameters: hubble_constant           = 0.703000030517578


In [44]:
# read constants from yt

a_exp = ds["aexp"]
H0 = ds["H0"] * const.km / const.Mpc
Omega_m0 = ds["omega_m"]
Omega_L0 = ds["omega_l"]
Omega_k0 = ds["omega_k"]
Omega_b0 = ds["omega_b"]

length_unit = ds["unit_l"]
density_unit = ds["unit_d"]
time_unit = ds["unit_t"]

mass_unit = density_unit * length_unit**3
vel_unit = length_unit / time_unit
energy_unit = mass_unit * vel_unit**2
energy_density_unit = density_unit * vel_unit**2

amr_level_sim_max = int(ds.parameters["levelmax"])
amr_level_course = int(np.log2(ds.domain_dimensions[0]))
amr_level_reduce_exp = min(-4, int(np.floor(np.log2(a_exp))))
amr_level_max = amr_level_sim_max - amr_level_course + amr_level_reduce_exp
print("Maximum AMR level: %d" % amr_level_max)

halo.convert_to_cgs(length_unit, density_unit)


Maximum AMR level: 13


In [47]:
# define size and resolution of the region of interest

# small grid
amr_level = 13
box_size = 2 * const.kpc

# medium grid
#amr_level = 10
#box_size = 20 * const.kpc

# large grid
#amr_level = 10
#box_size = 50 * const.kpc

left_edge = halo.coord - box_size / 2
N = int(box_size / length_unit * ds.domain_dimensions[0] * 2**amr_level)
lowres = 2**(amr_level_max - amr_level)

print("Grid size: %d" % N)
print("Lowres factor: %d" % lowres)


Grid size: 229
Lowres factor: 1


In [48]:
# get raw yt data from the region of interest

smooth_grid = True

if smooth_grid:
    data = ds.smoothed_covering_grid(level=amr_level, left_edge=(left_edge / length_unit), dims=[N]*3)
else:
    data = ds.covering_grid(level=amr_level, left_edge=(left_edge / length_unit), dims=[N]*3)


In [49]:
# read particle data from yt

particle_type = data["io", "particle_family"].value.astype(int)
is_dm = particle_type == DM
is_star = particle_type == STAR

coord_dm = (np.array([data["io", "particle_position_x"].value[is_dm], data["io", "particle_position_y"].value[is_dm], data["io", "particle_position_z"].value[is_dm]])) * length_unit - left_edge[:, None] - box_size / 2
mass_dm = data["io", "particle_mass"][is_dm] * mass_unit

coord_star = (np.array([data["io", "particle_position_x"].value[is_star], data["io", "particle_position_y"].value[is_star], data["io", "particle_position_z"].value[is_star]])) * length_unit - left_edge[:, None] - box_size / 2
mass_star = data["io", "particle_mass"][is_star] * mass_unit
tau_starbirth = data["io", "conformal_birth_time"][is_star].value


yt : [WARNING  ] 2023-10-27 16:31:12,777 The DM particle filter already exists. Overriding.
yt : [WARNING  ] 2023-10-27 16:31:12,778 The star particle filter already exists. Overriding.
yt : [WARNING  ] 2023-10-27 16:31:12,778 The cloud particle filter already exists. Overriding.
yt : [WARNING  ] 2023-10-27 16:31:12,779 The dust particle filter already exists. Overriding.
yt : [WARNING  ] 2023-10-27 16:31:12,779 The star_tracer particle filter already exists. Overriding.
yt : [WARNING  ] 2023-10-27 16:31:12,779 The cloud_tracer particle filter already exists. Overriding.
yt : [WARNING  ] 2023-10-27 16:31:12,779 The dust_tracer particle filter already exists. Overriding.
yt : [WARNING  ] 2023-10-27 16:31:12,780 The gas_tracer particle filter already exists. Overriding.
yt : [INFO     ] 2023-10-27 16:31:12,780 Adding particle_type: DM
yt : [INFO     ] 2023-10-27 16:31:12,790 Adding particle_type: star
yt : [INFO     ] 2023-10-27 16:31:12,800 Adding particle_type: cloud
yt : [INFO     ] 2

In [50]:
# read gas data from yt

coord = (np.array([data["index", "x"].value, data["index", "y"].value, data["index", "z"].value])) * length_unit - left_edge[:, None, None, None] - box_size / 2
density = data["ramses", "Density"].value * density_unit
metallicity = data["ramses", "Metallicity"].value 
pressure = data["ramses", "Pressure"].value * energy_density_unit
energy_turb = data["ramses", "hydro_scalar_01"].value * vel_unit**2
ref_crit = data["ramses", "hydro_scalar_02"].value
vel_vec = np.array([data["ramses", "x-velocity"].value, data["ramses", "y-velocity"].value, data["ramses", "z-velocity"].value]) * vel_unit


In [51]:
grid_name = "data_d%d_l%d_b%d" % (dump, lowres, int(box_size / const.kpc))
print("filename: %s.npz" % grid_name)


filename: data_d8_l1_b2.npz


In [54]:
safe_savez(
    grid_name,
    halo_idx=halo.idx,
    halo_mass=halo.mass,
    a_exp=a_exp,
    H0=H0,
    Omega_m0=Omega_m0,
    Omega_L0=Omega_L0,
    Omega_k0=Omega_k0,
    Omega_b0=Omega_b0,
    length_unit=length_unit,
    density_unit=density_unit,
    time_unit=time_unit,
    mass_unit=mass_unit,
    vel_unit=vel_unit,
    energy_unit=energy_unit,
    energy_density_unit=energy_density_unit,
    amr_level=amr_level,
    box_size=box_size,
    left_edge=left_edge,
    N=N,
    coord_dm=coord_dm,
    mass_dm=mass_dm,
    coord_star=coord_star,
    mass_star=mass_star,
    tau_starbirth=tau_starbirth,
    coord=coord,
    density=density,
    metallicity=metallicity,
    pressure=pressure,
    energy_turb=energy_turb,
    ref_crit=ref_crit,
    vel_vec=vel_vec,
    lowres=lowres
)
